In [120]:
import pandas as pd
import cv2#reading and resizing                 
import numpy as np#arrays         
import os#dealing with directories                  
from random import shuffle #to shuffle data
from tqdm import tqdm#loop progress bar  
from sklearn.metrics import roc_auc_score    
import matplotlib.pyplot as plt # for visualizations
import tensorflow as tf # For tensor operations
import zipfile
import os, sys
from PIL import Image

In [121]:
df = pd.read_csv(r"dataset.csv")
del df["Unnamed: 0"]

In [122]:
df.head()

,image,target
0,0-0.jpg,0
1,0-1.jpg,0
2,0-2.jpg,3
3,0-3.jpg,0
4,0-4.jpg,0


In [123]:
def change(x):
    if x==0:
        return 0
    else:
        return 1
    
df.target = df.target.apply(lambda x: change(x))

In [124]:
df.target.value_counts()

0    2556
1     763
Name: target, dtype: int64

In [125]:
for i in df.target.value_counts():
    print((i/len(df)*100))

77.01114793612534
22.988852063874663


In [126]:
from keras.preprocessing import image
train_image = []
for i in tqdm(range(df.shape[0])):
    img = image.load_img(r'../images/'+df['image'][i], target_size=(150,150,1), grayscale=False)
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
X = np.array(train_image)

 64%|██████████████████████████████████████████████████                            | 2130/3319 [00:58<01:26, 13.70it/s]C:\Anaconda3\lib\site-packages\PIL\Image.py:989: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
100%|██████████████████████████████████████████████████████████████████████████████| 3319/3319 [01:06<00:00, 49.71it/s]


In [151]:
from keras.utils import to_categorical 
vals=df['target'].values
y = to_categorical(vals)

In [152]:
y

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [129]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2,stratify=y)
X_test, X_val, y_test, y_val  = train_test_split(X_test, y_test, test_size=0.1, random_state=1)

In [153]:
X.shape,y.shape

((3319, 150, 150, 3), (3319, 2))

In [130]:
X_train.shape,X_test.shape,X_val.shape

((2655, 150, 150, 3), (597, 150, 150, 3), (67, 150, 150, 3))

In [134]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm

In [132]:
# model = Sequential()
# model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(28,28,3)))
# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
# model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(2, activation='sigmoid'))
# model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

In [139]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150,150,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [140]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 148, 148, 32)      896       
_________________________________________________________________
activation_6 (Activation)    (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_7 (Activation)    (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 34, 34, 64)        18496     
__________

In [154]:
history = model.fit(X, y, epochs=20)

Epoch 1/20
3319/3319 [==============================] - 70s 21ms/step - loss: 0.4677 - acc: 0.7898
Epoch 2/20
3319/3319 [==============================] - 68s 21ms/step - loss: 0.4591 - acc: 0.7917
Epoch 3/20
3319/3319 [==============================] - 64s 19ms/step - loss: 0.4474 - acc: 0.7942
Epoch 4/20
3319/3319 [==============================] - 66s 20ms/step - loss: 0.4390 - acc: 0.7977
Epoch 5/20
3319/3319 [==============================] - 67s 20ms/step - loss: 0.4300 - acc: 0.7984
Epoch 6/20
3319/3319 [==============================] - 75s 23ms/step - loss: 0.4176 - acc: 0.8043
Epoch 7/20
3319/3319 [==============================] - 73s 22ms/step - loss: 0.4122 - acc: 0.8170
Epoch 8/20
3319/3319 [==============================] - 74s 22ms/step - loss: 0.4066 - acc: 0.8233
Epoch 9/20
3319/3319 [==============================] - 61s 18ms/step - loss: 0.3830 - acc: 0.8280
Epoch 10/20
3319/3319 [==============================] - 54s 16ms/step - loss: 0.3962 - acc: 0.8146
Epoch 11/

In [155]:
import statistics 
statistics.mean(history.history['acc'])

0.8252561012496787

In [156]:
model.save_weights('infant-3.h5')

In [157]:
# making predictions
prediction = model.predict_classes(X_val)

In [145]:
# prediction

In [146]:
# preds=[]
# decoder = {0:'Absent', 1:'Present'}

# for pred in prediction:
#      preds.append(decoder[pred])
# len(preds)

In [147]:
from numpy import argmax
y_val = argmax(y_val,axis =1)

In [148]:
correct =0
for pred,y in zip(prediction,y_val):
    if pred == y:
        correct+=1
print(correct/len(preds))

0.7014925373134329


In [173]:
def predit_type(img_path):
    import keras

    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(150,150,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2))
    model.add(Activation('sigmoid'))
    model.load_weights("infant-3.h5")

    model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])


    img = image.load_img(img_path, target_size=(150,150,1), grayscale=False)
    img = image.img_to_array(img)
    img = img/255
    new_X = np.array(img).reshape(1,150,150,3)

    pred = model.predict_classes(new_X)
    return pred[0]

In [175]:
img_path = r"..\images\2-0.jpg"
predit_type(img_path)

0